In [2]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>

<article class="day-desc read-aloud"><h2>--- Day 9: Explosives in Cyberspace ---</h2><p>Wandering around a secure area, you come across a datalink port to a new part of the network. After briefly scanning it for interesting files, you find one file in particular that catches your attention. It's compressed with an experimental format, but fortunately, the documentation for the format is nearby.</p>
<p>The format compresses a sequence of characters. Whitespace is ignored. To indicate that some sequence should be repeated, a marker is added to the file, like <code>(10x2)</code>. To decompress this marker, take the subsequent <code>10</code> characters and repeat them <code>2</code> times. Then, continue reading the file <em>after</em> the repeated data.  The marker itself is not included in the decompressed output.</p>
<p>If parentheses or other characters appear within the data referenced by a marker, that's okay - treat it like normal data, not a marker, and then resume looking for markers after the decompressed section.</p>
<p>For example:</p>
<ul>
<li><code>ADVENT</code> contains no markers and decompresses to itself with no changes, resulting in a decompressed length of <code>6</code>.</li>
<li><code>A(1x5)BC</code> repeats only the <code>B</code> a total of <code>5</code> times, becoming <code>ABBBBBC</code> for a decompressed length of <code>7</code>.</li>
<li><code>(3x3)XYZ</code> becomes <code>XYZXYZXYZ</code> for a decompressed length of <code>9</code>.</li>
<li><code>A(2x2)BCD(2x2)EFG</code> doubles the <code>BC</code> and <code>EF</code>, becoming <code>ABCBCDEFEFG</code> for a decompressed length of <code>11</code>.</li>
<li><code>(6x1)(1x3)A</code> simply becomes <code>(1x3)A</code> - the <code>(1x3)</code> looks like a marker, but because it's within a data section of another marker, it is not treated any differently from the <code>A</code> that comes after it. It has a decompressed length of <code>6</code>.</li>
<li><code>X(8x2)(3x3)ABCY</code> becomes <code>X(3x3)ABC(3x3)ABCY</code> (for a decompressed length of <code>18</code>), because the decompressed data from the <code>(8x2)</code> marker (the <code>(3x3)ABC</code>) is skipped and not processed further.</li>
</ul>
<p>What is the <em>decompressed length</em> of the file (your puzzle input)? Don't count whitespace.</p>
</article>


In [3]:
tests = [
    {
        "name": "ADVENT contains no markers and decompresses to itself with no changes, resulting in a decompressed length of 6.",
        "chrs": "ADVENT",
        "expected": 6,
    },
    {
        "name": "A(1x5)BC repeats only the B a total of 5 times, becoming ABBBBBC for a decompressed length of 7.",
        "chrs": "A(1x5)BC",
        "expected": 7,
    },
    {
        "name": "(3x3)XYZ becomes XYZXYZXYZ for a decompressed length of 9.",
        "chrs": "(3x3)XYZ",
        "expected": 9,
    },
    {
        "name": "A(2x2)BCD(2x2)EFG doubles the BC and EF, becoming ABCBCDEFEFG for a decompressed length of 11.",
        "chrs": "A(2x2)BCD(2x2)EFG",
        "expected": 11,
    },
    {
        "name": "A(2x2)BCD(2x2)EFG doubles the BC and EF, becoming ABCBCDEFEFG for a decompressed length of 11.",
        "chrs": "A(2x2)BCD(2x2)EFG",
        "expected": 11,
    },
    {
        "name": "(6x1)(1x3) A simply becomes (1x3)A - the (1x3) looks like a marker, but because it's within a data section of another marker, it is not treated any differently from the A that comes after it. It has a decompressed length of 6.",
        "chrs": "(6x1)(1x3) A",
        "expected": 6,
    },
    {
        "name": "X(8x2)(3x3)ABCY becomes X(3x3)ABC(3x3)ABCY (for a decompressed length of 18), because the decompressed data from the (8x2) marker (the (3x3)ABC) is skipped and not processed further.",
        "chrs": "X(8x2)(3x3)ABCY",
        "expected": 18,
    },
]


def decompressedI_length(chrs: str) -> int:
    chrs = chrs.replace(" ", "")
    decompressed = []
    last = 0

    for m in re.finditer(r"\((\d+)x(\d+)\)", chrs):
        if m.start() > last:
            decompressed.append(chrs[last : m.start()])

        if m.start() >= last:
            start = m.end()
            length, rep = (int(i) for i in chrs[m.start() + 1 : m.end() - 1].split("x"))

            decompressed.append((chrs[start : start + length] * rep))

            last = start + length

    decompressed.append(chrs[last:])
    return len("".join(decompressed))


run_tests_params(decompressedI_length, tests)


Test ADVENT contains no markers and decompresses to itself with no changes, resulting in a decompressed length of 6. passed, for decompressedI_length.
Test A(1x5)BC repeats only the B a total of 5 times, becoming ABBBBBC for a decompressed length of 7. passed, for decompressedI_length.
Test (3x3)XYZ becomes XYZXYZXYZ for a decompressed length of 9. passed, for decompressedI_length.
Test A(2x2)BCD(2x2)EFG doubles the BC and EF, becoming ABCBCDEFEFG for a decompressed length of 11. passed, for decompressedI_length.
Test A(2x2)BCD(2x2)EFG doubles the BC and EF, becoming ABCBCDEFEFG for a decompressed length of 11. passed, for decompressedI_length.
Test (6x1)(1x3) A simply becomes (1x3)A - the (1x3) looks like a marker, but because it's within a data section of another marker, it is not treated any differently from the A that comes after it. It has a decompressed length of 6. passed, for decompressedI_length.
Test X(8x2)(3x3)ABCY becomes X(3x3)ABC(3x3)ABCY (for a decompressed length of 18

In [4]:
decompressedI_length(
    "ADVENTA(1x5)BC(3x3)XYZA(2x2)BCD(2x2)EFG(6x1)(1x3)AX(8x2)(3x3)ABCY"
)

57

In [5]:
with open("../input/day9.txt") as f:
    print("Part I:", decompressedI_length(f.read().strip()))

# 399435 too high

Part I: 107035


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>107035</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>Apparently, the file actually uses <em>version two</em> of the format.</p>
<p>In version two, the only difference is that markers within decompressed data <em>are</em> decompressed. This, the documentation explains, provides much more substantial compression capabilities, allowing many-gigabyte files to be stored in <span title="&quot;It's the bomb!&quot;, the documentation claims.">only a few kilobytes</span>.</p>
<p>For example:</p>
<ul>
<li><code>(3x3)XYZ</code> still becomes <code>XYZXYZXYZ</code>, as the decompressed section contains no markers.</li>
<li><code>X(8x2)(3x3)ABCY</code> becomes <code>XABCABCABCABCABCABCY</code>, because the decompressed data from the <code>(8x2)</code> marker is then further decompressed, thus triggering the <code>(3x3)</code> marker twice for a total of six <code>ABC</code> sequences.</li>
<li><code>(27x12)(20x12)(13x14)(7x10)(1x12)A</code> decompresses into a string of <code>A</code> repeated <code>241920</code> times.</li>
<li><code>(25x3)(3x3)ABC(2x3)XY(5x2)PQRSTX(18x9)(3x2)TWO(5x7)SEVEN</code> becomes <code>445</code> characters long.</li>
</ul>
<p>Unfortunately, the computer you brought probably doesn't have enough memory to actually decompress the file; you'll have to <em>come up with another way</em> to get its decompressed length.</p>
<p>What is the <em>decompressed length</em> of the file using this improved format?</p>
</article>

</main>


In [8]:
tests = [
    {
        "name": "(3x3)XYZ still becomes XYZXYZXYZ, as the decompressed section contains no markers.",
        "chrs": "(3x3)XYZ",
        "expected": len("XYZXYZXYZ"),
    },
    {
        "name": "X(8x2)(3x3)ABCY becomes XABCABCABCABCABCABCY, because the decompressed data from the (8x2) marker is then further decompressed, thus triggering the (3x3) marker twice for a total of six ABC sequences.",
        "chrs": "X(8x2)(3x3)ABCY",
        "expected": len("XABCABCABCABCABCABCY"),
    },
    {
        "name": "(27x12)(20x12)(13x14)(7x10)(1x12)A decompresses into a string of A repeated 241920 times.",
        "chrs": "(27x12)(20x12)(13x14)(7x10)(1x12)A",
        "expected": 241920,
    },
    {
        "name": "(25x3)(3x3)ABC(2x3)XY(5x2)PQRSTX(18x9)(3x2)TWO(5x7)SEVEN becomes 445 characters long.",
        "chrs": "(25x3)(3x3)ABC(2x3)XY(5x2)PQRSTX(18x9)(3x2)TWO(5x7)SEVEN",
        "expected": 445,
    },
]


def decompressed(chrs: str) -> int:
    if not chrs:
        return 0

    m = re.search(r"\((\d+)x(\d+)\)", chrs)
    if not m:
        return len(chrs)

    length, rep = (int(i) for i in chrs[m.start() + 1 : m.end() - 1].split("x"))
    return (
        m.start()
        + decompressed(chrs[m.end() : m.end() + length]) * rep
        + decompressed(chrs[m.end() + length :])
    )


def decompressedII_length(chrs: str) -> int:
    return decompressed(chrs.strip().replace(" ", ""))


run_tests_params(decompressedII_length, tests)


Test (3x3)XYZ still becomes XYZXYZXYZ, as the decompressed section contains no markers. passed, for decompressedII_length.
Test X(8x2)(3x3)ABCY becomes XABCABCABCABCABCABCY, because the decompressed data from the (8x2) marker is then further decompressed, thus triggering the (3x3) marker twice for a total of six ABC sequences. passed, for decompressedII_length.
Test (27x12)(20x12)(13x14)(7x10)(1x12)A decompresses into a string of A repeated 241920 times. passed, for decompressedII_length.
Test (25x3)(3x3)ABC(2x3)XY(5x2)PQRSTX(18x9)(3x2)TWO(5x7)SEVEN becomes 445 characters long. passed, for decompressedII_length.
Success


In [9]:
with open("../input/day9.txt") as f:
    print("Part II:", decompressedII_length(f.read()))

Part II: 11451628995


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>11451628995</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>
<p>At this point, you should <a href="/2016">return to your Advent calendar</a> and try another puzzle.</p>
<p>If you still want to see it, you can <a href="9/input" target="_blank">get your puzzle input</a>.</p>
<p>You can also <span class="share">[Share<span class="share-content">on
  <a href="https://twitter.com/intent/tweet?text=I%27ve+completed+%22Explosives+in+Cyberspace%22+%2D+Day+9+%2D+Advent+of+Code+2016&amp;url=https%3A%2F%2Fadventofcode%2Ecom%2F2016%2Fday%2F9&amp;related=ericwastl&amp;hashtags=AdventOfCode" target="_blank">Twitter</a>
  <a href="javascript:void(0);" onclick="var ms; try{ms=localStorage.getItem('mastodon.server')}finally{} if(typeof ms!=='string')ms=''; ms=prompt('Mastodon Server?',ms); if(typeof ms==='string' &amp;&amp; ms.length){this.href='https://'+ms+'/share?text=I%27ve+completed+%22Explosives+in+Cyberspace%22+%2D+Day+9+%2D+Advent+of+Code+2016+%23AdventOfCode+https%3A%2F%2Fadventofcode%2Ecom%2F2016%2Fday%2F9';try{localStorage.setItem('mastodon.server',ms);}finally{}}else{return false;}" target="_blank">Mastodon</a></span>]</span> this puzzle.</p>
</main>
